In [1]:
import pandas as pd
import numpy as np

<div class="alert alert-block alert-warning">
    <h1>Rendu TP optimisation discrète L3-ISUP 2023-2024</h1>
     <hr style = "height:1px; border-width:0; color:gray; background-color:gray"/>
    <h2>Sommaire</h2>
    <div><h4>Importation et traitement des données (villes, métro)</h4>
        <ul>
            <li>Importation des datasets (métro, villes)</li>
            <li>Création des fonctions de prétraitement des données</li>
            <li>Création des fonctions de changement de forme des données (matrice d'adjacence et liste d'adjacence)</li>
        </ul>
    </div>
    <div><h4>Algorithmes de plus court chemins</h4>
        <ul>
            <li>Demoucron floyd</li>
            <li>Dijkstra</li>
        </ul>
    </div>
    <div><h4>Algorithme d'affectation</h4>
        <ul>
            <li>Méthode hongroise</li>
        </ul>
    </div>
    <div><h4>Heuristiques du voyageur de commerce</h4>
        <ul>
            <li>Méthode du plus proche voisin</li>
            <li>Méthode de l'arc de regret maximal</li>
            <li>Méthode de l'insertion de cout minimal</li>
        </ul>
    </div>
</div>

<div class="alert alert-block alert-warning"><h2>I - Importation et traitement des données</h2>
    <ol>
        <li>Données des stations du métro parisien</li>
        <li>Données des 34 villes</li>
    </ol>
</div>

### Données stations de métro

In [2]:
f = open("data/Metro_Paris.csv")
data_metro = pd.read_csv(f,  delimiter = ";")
print(f'Dataset metro shape : {data_metro.shape}')
data_metro.head(10)

Dataset metro shape : (395, 2)


,Numéro ligne,Stations
0,1,La Défense (Grande Arche)
1,1,Esplanade de la Défense
2,1,Pont de Neuilly
3,1,Les Sablons
4,1,Porte Maillot
5,1,Argentine
6,1,Charles de Gaulle - Etoile
7,1,George V
8,1,Franklin D. Roosevelt
9,1,Champs-Elysées - Clemenceau


<div class="alert alert-block alert-info">
    <h4>Cette fonction permet de modifier la formes des donnnées dans notre cas créer la matrice d'adjacence ainsi que la liste d'adjacence du graphe</h4>
    <hr style = "height:0.7px; border-width:0; color:gray; background-color:gray"/>
    <p>L'algorithme prend en paramètre le dataset pandas ainsi que deux variables "corres_time" et "stay_time" qui correspondent respectivement au temps pour changer de ligne et au temps pour passer d'une station à l'autre</p>
</div>

In [3]:
def Graph_metro(data, corres_time = 5, stay_time = 1):
    """ The function return the adjency list and the adjency matrix of the graph
    
    Input : - data : pandas dataframe of Paris's underground's stations
            - corres_time : time needed to change lines
            - stay_time : time needed to go from one station to the next one on the same line
             
    Output(tuple) : - adjency_list : Adjency list dict(dict)
                    - data_adjancy : pandas datafram de la matrice d'adjacence
                    - dict_corres : Dict of all correspondance from any stations
        """
    lines = data['Numéro ligne'].unique()
    stations = data['Stations'].unique()
    adjency_list = {}
    dict_lines_stations = {}
    dict_corres = {}
    
    for line in lines:
        
        # dict_lines_stations[line] = [all stations on line 'line']
        dict_lines_stations[line] = np.array(data[data['Numéro ligne'] == str(line)]['Stations'])
        
        # Number stations on line 'line'
        nb_stations = len(dict_lines_stations[line])
        
        for i in range(nb_stations) :
            dict_corres[dict_lines_stations[line][i]] = []
            adjency_list[dict_lines_stations[line][i] + str(line)] = {}
            
            # Storing of all the correspondences for the station i
            correspondences = data[(data['Numéro ligne'] != str(line)) & (data['Stations'] == dict_lines_stations[line][i])]
            
            for j in range(len(correspondences)) :
                adjency_list[dict_lines_stations[line][i] + str(line)][correspondences.values[j,1] + str(correspondences.values[j,0])] = corres_time
                dict_corres[dict_lines_stations[line][i]].append(correspondences.values[j,1] + str(correspondences.values[j,0]))
            
            # If the the station i is the start of line line
            if i == 0:
                adjency_list[dict_lines_stations[line][i] + str(line)][dict_lines_stations[line][i+1]+ str(line)] =  stay_time
                
            #If the station i is the terminus of line line    
            elif i == nb_stations - 1:
                adjency_list[dict_lines_stations[line][i] + str(line)][dict_lines_stations[line][i-1]+ str(line)] = stay_time
    
            else:
                adjency_list[dict_lines_stations[line][i] + str(line)][dict_lines_stations[line][i-1]+ str(line)] = stay_time
                adjency_list[dict_lines_stations[line][i] + str(line)][dict_lines_stations[line][i+1]+ str(line)] =  stay_time
        
    
    # Création de la matrice d'adjacence
    stations = adjency_list.keys()
    temp = np.ones((len(stations), len(stations)))*np.inf
    data_adjency = pd.DataFrame(temp, columns=stations, index=stations)
    for key in stations:
        for val in adjency_list[key]:
            data_adjency.loc[key, val] = adjency_list[key][val] 

            
    return adjency_list, data_adjency, dict_corres

In [4]:
Graph_metro(data_metro)[0]

{'La Défense (Grande Arche)1': {'Esplanade de la Défense1': 1},
 'Esplanade de la Défense1': {'La Défense (Grande Arche)1': 1,
  'Pont de Neuilly1': 1},
 'Pont de Neuilly1': {'Esplanade de la Défense1': 1, 'Les Sablons1': 1},
 'Les Sablons1': {'Pont de Neuilly1': 1, 'Porte Maillot1': 1},
 'Porte Maillot1': {'Les Sablons1': 1, 'Argentine1': 1},
 'Argentine1': {'Porte Maillot1': 1, 'Charles de Gaulle - Etoile1': 1},
 'Charles de Gaulle - Etoile1': {'Charles de Gaulle - Etoile2': 5,
  'Charles de Gaulle - Etoile6': 5,
  'Argentine1': 1,
  'George V1': 1},
 'George V1': {'Charles de Gaulle - Etoile1': 1, 'Franklin D. Roosevelt1': 1},
 'Franklin D. Roosevelt1': {'Franklin D. Roosevelt9': 5,
  'George V1': 1,
  'Champs-Elysées - Clemenceau1': 1},
 'Champs-Elysées - Clemenceau1': {'Champs-Elysées - Clemenceau13': 5,
  'Franklin D. Roosevelt1': 1,
  'Concorde1': 1},
 'Concorde1': {'Concorde8': 5,
  'Concorde12': 5,
  'Champs-Elysées - Clemenceau1': 1,
  'Tuileries1': 1},
 'Tuileries1': {'Conco

In [5]:
Graph_metro(data_metro)[1]

,La Défense (Grande Arche)1,Esplanade de la Défense1,Pont de Neuilly1,Les Sablons1,Porte Maillot1,Argentine1,Charles de Gaulle - Etoile1,George V1,Franklin D. Roosevelt1,Champs-Elysées - Clemenceau1,...,Pont Cardinet14,Saint-Lazare14,Madeleine14,Pyramides14,Châtelet14,Gare de Lyon14,Bercy14,Cour Saint-Emilion14,Bibliothèque François Mitterrand14,Olympiades14
La Défense (Grande Arche)1,inf,1.0,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
Esplanade de la Défense1,1.0,inf,1.0,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
Pont de Neuilly1,inf,1.0,inf,1.0,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
Les Sablons1,inf,inf,1.0,inf,1.0,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
Porte Maillot1,inf,inf,inf,1.0,inf,1.0,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Gare de Lyon14,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,1.0,inf,1.0,inf,inf,inf
Bercy14,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,1.0,inf,1.0,inf,inf
Cour Saint-Emilion14,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,1.0,inf,1.0,inf
Bibliothèque François Mitterrand14,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,1.0,inf,1.0


### Données 34 villes

In [6]:
f = open("data/Capitales_Europe.csv")
data_villes = pd.read_csv(f,  delimiter = ";")
data_villes = data_villes.drop(['villes'], axis = 1)

data_villes.head()

,Villes,Latitude,Longitude,Athenes,Belgrade,Berlin,Berne,Bratislava,Bruxelles,Bucarest,...,Reykjavik,Riga,Rome,Sarajevo,Sofia,Stockholm,Tallinn,Varsovie,Vienne,Zagreb
0,Athenes,"37,976","23,736",0,807,1804,1662,1251,2090,745,...,4163,2109,1052,791,526,2409,2387,1599,1283,1081
1,Belgrade,"44,813","20,463",807,0,1001,1034,451,1372,448,...,3377,1373,720,193,329,1622,1651,826,490,368
2,Berlin,"52,516","13,377",1804,1001,0,751,553,643,1295,...,2386,845,1182,1032,1319,812,1043,519,524,769
3,Berne,"46,948","7,44",1662,1034,751,0,737,492,1472,...,2613,1586,689,923,1336,1544,1794,1138,684,666
4,Bratislava,"48,149","17,107",1251,451,553,737,0,965,804,...,2927,1085,783,488,776,1245,1350,532,55,274


<div class="alert alert-block alert-info">
    <h4>Cette fonction permet de modifier le dataset des villes avec Pandas en changeant son index, en supprimant des colonnes afin de retourner un dataframe pandas similaire à une matrice d'adjacence</h4>
    <hr style = "height:0.7px; border-width:0; color:gray; background-color:gray"/>
    <p>L'algorithme prend en paramètre le dataset pandas</p>
</div>

In [7]:
def preprocessing_villes(df):
    """ Creation de la matrice d'adjacence pour les villes"""
    columns = list(df.columns)
    columns.remove('Latitude')
    columns.remove('Longitude')
    data_distances = df[columns]
    data_distances = data_distances.reset_index()
    data_distances = data_distances.reset_index(drop = True, inplace=False)
    data_distances = data_distances.set_index('Villes')
    data_distances = data_distances.drop(['index'], axis = 1)
    return data_distances

data_villes = preprocessing_villes(data_villes)

In [8]:
data_villes.head()

,Athenes,Belgrade,Berlin,Berne,Bratislava,Bruxelles,Bucarest,Budapest,Chisinau,Copenhague,...,Reykjavik,Riga,Rome,Sarajevo,Sofia,Stockholm,Tallinn,Varsovie,Vienne,Zagreb
Villes,,,,,,,,,,,,,,,,,,,,,
Athenes,0,807,1804,1662,1251,2090,745,1125,1089,2137,...,4163,2109,1052,791,526,2409,2387,1599,1283,1081
Belgrade,807,0,1001,1034,451,1372,448,318,692,1330,...,3377,1373,720,193,329,1622,1651,826,490,368
Berlin,1804,1001,0,751,553,643,1295,689,1263,355,...,2386,845,1182,1032,1319,812,1043,519,524,769
Berne,1662,1034,751,0,737,492,1472,878,1617,1033,...,2613,1586,689,923,1336,1544,1794,1138,684,666
Bratislava,1251,451,553,737,0,965,804,162,887,892,...,2927,1085,783,488,776,1245,1350,532,55,274


<div class="alert alert-block alert-info">
    <h4>Cette fonction permet de créer la liste d'adjacence du graphe du dataset des villes européennes</h4>
    <hr style = "height:0.7px; border-width:0; color:gray; background-color:gray"/>
    <p>L'algorithme prend en paramètre le dataset pandas ainsi qu'une variable "threshold" qui permet de supprimer les distances supérieurs à ce threshold</p>
</div>

In [9]:
def Graph_ville(data, threshold = 1000):
    """ Retourne la liste d'adjacence du graphe des ville europeene"""
    
    airport_names = list(data.columns)
    array_distance = np.array(data)
    array_distance = np.where((array_distance > threshold)|(array_distance == 0), np.inf, array_distance)
    
    list_adjency = {}
    for airport_start in airport_names:
        list_adjency[airport_start]= {}
        for airport_end in airport_names:
            if airport_end == airport_start:
                continue
            elif data[airport_start][airport_end] > threshold:
                continue
            else:
                list_adjency[airport_start][airport_end] = data[airport_start][airport_end]
    
    return list_adjency

In [10]:
Graph_ville(data_villes)

{'Athenes': {'Belgrade': 807,
  'Bucarest': 745,
  'La Valette': 851,
  'Nicosie': 914,
  'Podgorica': 624,
  'Sarajevo': 791,
  'Sofia': 526},
 'Belgrade': {'Athenes': 807,
  'Bratislava': 451,
  'Bucarest': 448,
  'Budapest': 318,
  'Chisinau': 692,
  'Ljubljana': 485,
  'Podgorica': 281,
  'Prague': 740,
  'Rome': 720,
  'Sarajevo': 193,
  'Sofia': 329,
  'Varsovie': 826,
  'Vienne': 490,
  'Zagreb': 368},
 'Berlin': {'Berne': 751,
  'Bratislava': 553,
  'Bruxelles': 643,
  'Budapest': 689,
  'Copenhague': 355,
  'La Haye': 617,
  'Londres': 929,
  'Ljubljana': 723,
  'Luxembourg': 592,
  'Oslo': 838,
  'Paris': 874,
  'Prague': 281,
  'Riga': 845,
  'Stockholm': 812,
  'Varsovie': 519,
  'Vienne': 524,
  'Zagreb': 769},
 'Berne': {'Berlin': 751,
  'Bratislava': 737,
  'Bruxelles': 492,
  'Budapest': 878,
  'La Haye': 614,
  'Londres': 747,
  'Ljubljana': 549,
  'Luxembourg': 330,
  'Paris': 434,
  'Prague': 621,
  'Rome': 689,
  'Sarajevo': 923,
  'Vienne': 684,
  'Zagreb': 666},
 

<div class="alert alert-block alert-warning"><h2>II - Algorithmes de plus court chemin</h2>
    <ol>
        <li>Demoucron floyd</li>
        <li>Dijkstra</li>
    </ol>
</div>

# Demoucron floyd

In [11]:
inf = np.inf
adjacency_matrix = np.array([[inf, 3, 8, 6, inf, inf],
                 [inf, inf, inf, 2, 6, inf],
                 [inf, inf, inf, inf, 1, inf],
                 [inf, inf, 2, inf, inf, 7],
                 [inf, inf, inf, inf, inf, 2],
                 [inf, inf, inf, inf, inf, inf]])

<div class="alert alert-block alert-info">
    <h4>Cette fonction permet d'afficher de manière plus visuelle une matrice</h4>
    <hr style = "height:0.7px; border-width:0; color:gray; background-color:gray"/>
    <p>L'algorithme prend en paramètre un numpy array</p>
</div>

In [12]:
def matrix_display(matrix):
    """Display the adjency matrix in a clear way"""
    for row in range(len(matrix)):
        for col in range(len(matrix)):
            print(f'{matrix[row][col]:<4}', end='  ')
        print()    

In [13]:
matrix_display(adjacency_matrix)

inf   3.0   8.0   6.0   inf   inf   
inf   inf   inf   2.0   6.0   inf   
inf   inf   inf   inf   1.0   inf   
inf   inf   2.0   inf   inf   7.0   
inf   inf   inf   inf   inf   2.0   
inf   inf   inf   inf   inf   inf   


<div class="alert alert-block alert-info">
    <h4>Cette fonction permet de retourner la matrice de distancié par l'algo de demoucron floyd</h4>
    <hr style = "height:0.7px; border-width:0; color:gray; background-color:gray"/>
    <p>L'algorithme prend en paramètre la matrice d'adjacence sous forme de numpy array</p>
    <p>Cette fonction vérifie qu'il n'y a pas de circuit absorbant. En effet a chaques itérations elle vérifie qu'il n'y ai pas de valeur négative sur la diagonale</p>
</div>

In [14]:
def Demoucron_matrix(matrix):
    """ This function return the Demoucron distances matrix
    Input : - matrix : Adjency matrix (np.array) of the graph 
    Output : - Demoucron_matrix : The Demoucron matrix (np.array)
    """
    Demoucron_matrix = matrix.copy()
    distances_dict = {}
    
    for intermediate_node in range(len(Demoucron_matrix)):
        for start_node in range(len(Demoucron_matrix)):
            for end_node in range(len(Demoucron_matrix)):
                if start_node == end_node:
                    Demoucron_matrix[start_node][start_node] = 0
                
    
                # Vérification de présence de circuits absorbant si présence retourne une erreur    
                if Demoucron_matrix[end_node][end_node] < 0:
                    print("circuit absorbant")
                    return -1
                    
                Demoucron_matrix[start_node][end_node] = min(Demoucron_matrix[start_node][end_node],
                                                             Demoucron_matrix[start_node][intermediate_node] + Demoucron_matrix[intermediate_node][end_node])
                
                    
    return Demoucron_matrix

#### Résultat sur l'exemple du cours

In [15]:
matrix_display(Demoucron_matrix(adjacency_matrix))

0.0   3.0   7.0   5.0   8.0   10.0  
inf   0.0   4.0   2.0   5.0   7.0   
inf   inf   0.0   inf   1.0   3.0   
inf   inf   2.0   0.0   3.0   5.0   
inf   inf   inf   inf   0.0   2.0   
inf   inf   inf   inf   inf   0.0   


<div class="alert alert-block alert-info">
    <h4>Cette fonction permet de retourner la le chemin le plus court entre deux noeuds a partir du distancié de demoucron</h4>
    <hr style = "height:0.7px; border-width:0; color:gray; background-color:gray"/>
    <p>L'algorithme prend en parametre la matrice d'adjacence du graphe ainsi qu'un noeud de départ et un noeud d'arrivé</p>
    <p>Le principe de l'algorithme est le suivant : </p>
    <ol>
        <li>On commence par traiter le noeud final, noeud d'arrivé</li>
        <li>On regarde les noeuds intermédiaires voisins (en arriere), qui sont les lignes qui ont une valeure différente de l'inf dans la colonne du noeud en cours de traitement</li>
        <li>On choisit le noeud intermédiaire tel : <br> 
        distance(depart, intermediaire) + distance(intermediaire, en cours de traitement) = distance(départ, en cours de traitement)<br>
            Cette opération est facile en effet nous somme dans un distancié, on calcule facilement les distance entre les noeuds.
        </li>
        <li>Le nouveau noeud à traiter est le noeud intermediaire choisi à l'étape précédente</li>
        <li>On boucle jusqu'a tomber sur le noeud de départ</li>
    </ol>
</div>

In [16]:
def Demoucron_path(adj, start, end, threshold = np.inf):
    """ This function return the shortest path between start and end using Demoucron algorithm
    Input : - adj : Adjency matrix (np.array) of the graph
            - start : The starting node
            - end : The ending node
            - threshold : The value for unattainable egdes
            
    Outut(tuple) : - path : The shortest path between start and end (list)  
                   - final_distance : The shortest distance between start and end 
            
    """
    finished = 0
    path = [end]
    node = end
    d = Demoucron_matrix(adj)
    print(d)
    
    # start, end at the same position
    if start == end:
        return 0 
    
    # unattainable si les deux noeud de départ et d'arrivé sont inatteignable
    if d[start,end] == threshold:
        return -1
    
    while(True):
  
        neighbors_j = np.where(adj[:, node] != threshold)[0]

        found = 0
        for n in neighbors_j:

            if(d[start,n] + adj[n,node] == d[start, node] and found == 0):
                path.append(n)
                found = 1
                break
        
        node = path[-1]
        
        if d[start, node] == 0:
            break
        
        # For problems
        if found == 0:
            return -2
    
    final_distance = d[start, end]
    for k in range(len(path)):
        path[k]+=1
    return path[::-1], final_distance

#### Résultat sur l'exemple du cours avec départ : v1 et arrivée : v5
On a donc un chemin [v1, v2, v4, v3, v5] avec un coût de 8

In [17]:
Demoucron_path(adjacency_matrix,1-1,5-1, np.inf)

[[ 0.  3.  7.  5.  8. 10.]
 [inf  0.  4.  2.  5.  7.]
 [inf inf  0. inf  1.  3.]
 [inf inf  2.  0.  3.  5.]
 [inf inf inf inf  0.  2.]
 [inf inf inf inf inf  0.]]


([1, 2, 4, 3, 5], 8.0)

### Application aux stations de métro ainsi qu'aux villes

<div class="alert alert-block alert-info">
    <h4>Cette fonction permet de retourner le plus court chemin entre deux stations de métro par l'algo de demoucron floyd</h4>
    <hr style = "height:0.7px; border-width:0; color:gray; background-color:gray"/>
    <p>L'algorithme prend en parametre le dataset du métro ainsi qu'un noeud de départ et un noeud d'arrivé</p>
    <p>Le principe de l'algorithme est le suivant : </p>
    <ol>
        <li>On doit dans un premier temps récuperer la matrice d'adjacence à l'aide de la fonction "graph_metro()"</li>
        <li>On doit ensuite créer la matrice du distancié et trouver le chemin le plus court entre s1 et s2 grace à la fonction "Demoucron_path()"
        </li>
    </ol>
</div>

In [18]:
def Demoucron_subway_path(data, s1, s2, corres_time = 5, stay_time = 1):
    """ This function return the shortest path between s1 and s2 as well as the distance using the Demoucron algorithm
    """
    adjency_dataframe = Graph_metro(data, corres_time = corres_time, stay_time = stay_time)[1]
    station_names = list(adjency_dataframe.columns)
    
    adjency_matrix = np.array(adjency_dataframe)
    start = station_names.index(s1)
    end = station_names.index(s2)
    
    indice_path  = Demoucron_path(adjency_matrix, start, end, np.inf)
    final_distance = indice_path[1]
    
    path = [station_names[indice_path[0][i]-1] for i in range(len(indice_path[0]))]
    
    return path, final_distance

#### Calcul du chemin le plus court entre Mairie d'Issy12 et Jussieu10 avec un cout de correspondance de 5
Cette fonction n'est pas optimal pour trouver le plus court chemin car en O(n^3).
On optient donc un chemin de coût 22

In [19]:
Demoucron_subway_path(data_metro, "Mairie d'Issy12",'Jussieu10', corres_time = 5, stay_time = 1)

[[ 0.  1.  2. ... 22. 23. 24.]
 [ 1.  0.  1. ... 21. 22. 23.]
 [ 2.  1.  0. ... 20. 21. 22.]
 ...
 [22. 21. 20. ...  0.  1.  2.]
 [23. 22. 21. ...  1.  0.  1.]
 [24. 23. 22. ...  2.  1.  0.]]


(["Mairie d'Issy12",
  'Corentin Celton12',
  'Porte de Versailles12',
  'Convention12',
  'Vaugirard12',
  'Volontaires12',
  'Pasteur12',
  'Falguière12',
  'Montparnasse Bienvenue12',
  'Notre-Dame des Champs12',
  'Rennes12',
  'Sèvres - Babylone12',
  'Sèvres - Babylone10',
  'Mabillon10',
  'Odéon10',
  'Cluny - La Sorbonne10',
  'Maubert - Mutualité10',
  'Cardinal Lemoine10',
  'Jussieu10'],
 22.0)

<div class="alert alert-block alert-info">
    <h4>Cette fonction permet de retourner le plus court chemin entre deux villes par l'algo de demoucron floyd</h4>
    <hr style = "height:0.7px; border-width:0; color:gray; background-color:gray"/>
    <p>L'algorithme prend en parametre le dataset des 34 villes ainsi qu'une ville de départ et une villes d'arrivé</p>
    <p>Le principe de l'algorithme est le suivant : </p>
    <ol>
        <li>On doit dans un premier temps récuperer la matrice d'adjacence donc transformer "data_distances" en np.array.
        On doit de plus mettre une valeur très grande (inf) sur les distances supérieurs à 1000 (threshold)</li>
        <li>On doit ensuite créer la matrice du distancié et trouver le chemin le plus court entre v1 et v2 grace à la fonction "Demoucron_path()"
        </li>
    </ol>
</div>

In [20]:
def Demoucron_city_path(data, v1, v2, threshold = 1000):
    col = list(data.columns)
    array_distance = np.array(data)
    array_distance = np.where((array_distance > threshold)|(array_distance == 0), np.inf, array_distance)
    
    i = col.index(v1)
    j = col.index(v2)
    
    chemin_temp  = Demoucron_path(array_distance,i,j, np.inf)
    if chemin_temp == 0:
        print("Les villes sont au même endroit")
        return
        
    if chemin_temp == -1:
        print("On ne peux pas aller de la ville 1  à la ville 2")
        return
    
    if chemin_temp == -2:
        print("Il y a un gros problème")
        return -1
    chem_villes = [col[chemin_temp[0][i]-1] for i in range(len(chemin_temp[0]))]
    return chem_villes, chemin_temp[1]

In [21]:
data_villes.head()

,Athenes,Belgrade,Berlin,Berne,Bratislava,Bruxelles,Bucarest,Budapest,Chisinau,Copenhague,...,Reykjavik,Riga,Rome,Sarajevo,Sofia,Stockholm,Tallinn,Varsovie,Vienne,Zagreb
Villes,,,,,,,,,,,,,,,,,,,,,
Athenes,0,807,1804,1662,1251,2090,745,1125,1089,2137,...,4163,2109,1052,791,526,2409,2387,1599,1283,1081
Belgrade,807,0,1001,1034,451,1372,448,318,692,1330,...,3377,1373,720,193,329,1622,1651,826,490,368
Berlin,1804,1001,0,751,553,643,1295,689,1263,355,...,2386,845,1182,1032,1319,812,1043,519,524,769
Berne,1662,1034,751,0,737,492,1472,878,1617,1033,...,2613,1586,689,923,1336,1544,1794,1138,684,666
Bratislava,1251,451,553,737,0,965,804,162,887,892,...,2927,1085,783,488,776,1245,1350,532,55,274


In [22]:
data_villes.columns

Index(['Athenes', 'Belgrade', 'Berlin', 'Berne', 'Bratislava', 'Bruxelles',
       'Bucarest', 'Budapest', 'Chisinau', 'Copenhague', 'Dublin', 'Helsinki',
       'La Haye', 'La Valette', 'Lisbonne', 'Londres', 'Ljubljana',
       'Luxembourg', 'Madrid', 'Nicosie', 'Oslo', 'Paris', 'Podgorica',
       'Prague', 'Reykjavik', 'Riga', 'Rome', 'Sarajevo', 'Sofia', 'Stockholm',
       'Tallinn', 'Varsovie', 'Vienne', 'Zagreb'],
      dtype='object')

#### Calcul du chemin le plus court entre Athenes et Stockholm
Cette fonction n'est pas optimal pour trouver le plus court chemin car en O(n^3).
On optient donc un chemin de coût 2443

In [23]:
Demoucron_city_path(data_villes, 'Athenes', 'Stockholm')

[[   0.  807. 1811. ... 1633. 1297. 1082.]
 [ 807.    0. 1004. ...  826.  490.  368.]
 [1811. 1004.    0. ...  519.  524.  769.]
 ...
 [1633.  826.  519. ...    0.  557.  803.]
 [1297.  490.  524. ...  557.    0.  268.]
 [1082.  368.  769. ...  803.  268.    0.]]


(['Athenes', 'Belgrade', 'Varsovie', 'Stockholm'], 2443.0)

# Dijkstra

In [24]:
adjacency_list = {'v11' : {'v12': 1.1, 'v21': 1},
                   'v12' : {'v13': 1.1, 'v22': 1.5},
                   'v13' : {'v14': 1.1, 'v23': 2},
                   'v14' : {'v24': 1},
                   'v21' : {'v31': 1,'v22': 2},
                   'v22' : {'v23': 2, 'v32': 1},
                   'v23' : {'v33': 2, 'v24': 2},
                   'v24' : {'v34': 4},
                   'v31' : {'v41': 1, 'v32': 2},
                   'v32' : {'v42':2, 'v33': 1},
                   'v33' : {'v43': 2, 'v34': 1},
                   'v34' : {'v44':1},
                   'v41' : {'v42': 1},
                   'v42' : {'v43': 1},
                   'v43' : {'v44': 1},
                   'v44' : {}
                  }

<div class="alert alert-block alert-info">
    <h4>Cette fonction retourne le chemin le plus court entre deux noeuds grace à l'algorithme de Dijskra</h4>
    <hr style = "height:0.7px; border-width:0; color:gray; background-color:gray"/>
    <p>L'algorithme prend en parametre la liste d'adjacence du graphe ainsi qu'un noeud de départ et un noeud d'arrivé</p>
    <p>Le principe de l'algorithme est le suivant : </p>
    <ol>
        <li>Vérifier qu'il n'y a pas de valuations négatives dans le graphe</li>
        <li>Créer une liste queue, une liste seen ainsi qu'un dictionnaire de distances qui vont enregistrer respectivement les nœuds ouverts, les nœuds déjà traités et la distance entre les nœuds et le nœud de départ</li>
        <li>On boucle tant que la liste de queue n'est pas vide</li>
        <li>On prend le dernier élément de la queue qui est celui ayant la distance la plus petite du nœud de départ. On le supprime de la queue avec la fonction "pop" de Python et on va le traiter</li>
        <li>On regarde tous les voisins de ce nœud, on met à jour les distances des voisins dans le dictionnaire de distances sous la forme (noeud, distance voisin depuis le départ), et on ajoute les voisins dans la queue sous la forme (voisin, distance du voisin depuis le départ)</li>
        <li>On trie la queue par distance dans l'ordre décroissant la distance la plus petite à la fin</li>
        <li>On change le nœud à traiter en prenant le dernier élément qui est donc celui de distance minimale avec la fonction "pop()"</li>
        <li>Et on boucle</li>
    </ol>
    <p>La phase suivant est de reconstruire le chemin le plus court par backtracking. On part du noeud d'arriver et on remonte jusqu'au noeud de départ dans le dictionnaire de distances</p>
</div>

In [25]:
def Dijkstra_path(adj, start, end, verbose = False):
    """ This function return the shortest path between start and end using Dijkstra algorithm
    Input : - ajd : Adjacency list (dict(dict)) of the graph
            - start : The starting node
            - end : The ending node
            
    Output(tuple) : - path : The shortest path between start and end (list)  
                   - final_distance : The shortest distance between start and end 
    """
    
    # Vérification si présence de valuation négatif
    assert(np.array([( adj[i][j][1] > 0 for j in adj[i] ) for i in adj.keys()]).all())
    
    seen = []
    distances = {node: (None, np.inf) for node in adj}
    distances[start] = (start,0)
    
    queue = [(start, 0)]
    
    while queue:
        # store and delete the last element (the smallest)
        node, _ = queue.pop()
             
        if node not in seen:
            seen.append(node)
            node_dist = distances[node][1]
            
            for neighbor in adj[node]:
                neighbor_dist = node_dist + adj[node][neighbor]
                
                if neighbor_dist < distances[neighbor][1]:
                    distances[neighbor] = (node, neighbor_dist)
                    queue.append((neighbor, neighbor_dist))
        
        # sorting the list in descending order the smallest at the end
        queue.sort(key = lambda x: x[1], reverse=True) 
    
    # Backtracking
    node_ = end
    path = [end]
    final_dist = distances[node_][1]
    while node_ != start:
        if node_ not in distances:
            print(f"Il n'y a pas de chemin entre {start} et {end} ")
            return 0
        node_ = distances[node_][0]
        path.append(node_)
    
    path.reverse()
    if verbose:
        print(distances)

    return path, final_dist

In [26]:
Dijkstra_path(adjacency_list, 'v44', 'v24', verbose=False)

Il n'y a pas de chemin entre v44 et v24 


0

In [27]:
Dijkstra_path(adjacency_list, 'v11', 'v44', verbose=True)

{'v11': ('v11', 0), 'v12': ('v11', 1.1), 'v13': ('v12', 2.2), 'v14': ('v13', 3.3000000000000003), 'v21': ('v11', 1), 'v22': ('v12', 2.6), 'v23': ('v13', 4.2), 'v24': ('v14', 4.300000000000001), 'v31': ('v21', 2), 'v32': ('v22', 3.6), 'v33': ('v32', 4.6), 'v34': ('v33', 5.6), 'v41': ('v31', 3), 'v42': ('v41', 4), 'v43': ('v42', 5), 'v44': ('v43', 6)}


(['v11', 'v21', 'v31', 'v41', 'v42', 'v43', 'v44'], 6)

### Application aux stations de métro ainsi qu'aux villes

<div class="alert alert-block alert-info">
    <h4>Cette fonction est une fonction user friendly de l'algo Dijkstra appliqué aux stations du métro parisien</h4>
    <hr style = "height:0.7px; border-width:0; color:gray; background-color:gray"/>
    <p>L'algorithme prend en parametre le dataset du métro parisien</p>
    <p>Le principe de l'algorithme est le suivant : </p>
    <ol>
        <li>On initialise le temps de correspondance à 6 min et le temps pour aller d'une station à l'autre sur la même ligne à 1 min ce qui est le temps moyen pour approcher la réalité</li>
        <li>On demande à l'utilisateur de choisir la station de départ et la station d'arrivé</li>
        <li>On applique l'algo de dijkstra avec les donnée de l'utilisateur</li>
    </ol>
</div>

In [28]:
def Dijkstra_subway_path_user(data):
    
    corres_time = 6
    stay_time = 1.5
    
    stations = list(data['Stations'].unique())
    print(stations)
    
    start = input("Veuillez saisir la station de départ : ")
    adjency_list, data_adjency, dict_corres = Graph_metro(data, corres_time = corres_time, stay_time = stay_time)
    end = input("Veuillez saisir la station d'arrivée : ")
    
    
    if dict_corres[start] != []:
        start = dict_corres[start][0]
    else:
        start += str(np.array(data[data['Stations'] == str(start)]['Numéro ligne'])[0])
        

    if dict_corres[end] != []:
        end = dict_corres[end][0]
    else:
        end += str(np.array(data[data['Stations'] == str(end)]['Numéro ligne'])[0])
        
        
    return Dijkstra_path(adjency_list,start,end, verbose=False)

In [29]:
Dijkstra_subway_path_user(data_metro)

['La Défense (Grande Arche)', 'Esplanade de la Défense', 'Pont de Neuilly', 'Les Sablons', 'Porte Maillot', 'Argentine', 'Charles de Gaulle - Etoile', 'George V', 'Franklin D. Roosevelt', 'Champs-Elysées - Clemenceau', 'Concorde', 'Tuileries', 'Palais Royal - Musée du Louvre', 'Louvre - Rivoli', 'Châtelet', 'Hôtel de Ville', 'Saint-Paul (Le Marais)', 'Bastille', 'Gare de Lyon', 'Reuilly - Diderot', 'Nation', 'Porte de Vincennes', 'Saint-Mandé', 'Bérault', 'Château de Vincennes', 'Porte Dauphine', 'Victor Hugo', 'Ternes', 'Courcelles', 'Monceau', 'Villiers', 'Rome', 'Place de Clichy', 'Blanche', 'Pigalle', 'Anvers', 'Barbès - Rochechouart', 'La Chapelle', 'Stalingrad', 'Jaurès', 'Colonel Fabien', 'Belleville', 'Couronnes', 'Ménilmontant', 'Père Lachaise', 'Philippe Auguste', 'Alexandre Dumas', 'Avron', 'Pont de Levallois - Bécon', 'Anatole France', 'Louise Michel', 'Porte de Champerret', 'Pereire', 'Wagram', 'Malesherbes', 'Europe', 'Saint-Lazare', 'Havre-Caumartin', 'Opéra', 'Quatre Se

(["Mairie d'Issy12",
  'Corentin Celton12',
  'Porte de Versailles12',
  'Convention12',
  'Vaugirard12',
  'Volontaires12',
  'Pasteur12',
  'Falguière12',
  'Montparnasse Bienvenue12',
  'Notre-Dame des Champs12',
  'Rennes12',
  'Sèvres - Babylone12',
  'Sèvres - Babylone10',
  'Mabillon10',
  'Odéon10',
  'Cluny - La Sorbonne10',
  'Maubert - Mutualité10',
  'Cardinal Lemoine10',
  'Jussieu10',
  'Jussieu7'],
 37.5)

<div class="alert alert-block alert-info">
    <h4>Cette fonction est une fonction user friendly de l'algo Dijkstra appliqué aux 34 villes</h4>
    <hr style = "height:0.7px; border-width:0; color:gray; background-color:gray"/>
    <p>L'algorithme prend en parametre le dataset des villes européennes</p>
    <p>Le principe de l'algorithme est le suivant : </p>
    <ol>
        <li>On récupère la liste d'adjacence grace à la fonction "Graph_ville()"</li>
        <li>On demande à l'utilisateur de choisir l'aéroport de départ et celui d'arrivé</li>
        <li>On applique l'algo de dijkstra avec les données de l'utilisateur</li>
    </ol>
</div>

In [30]:
def Dijkstra_city_path_user(data):
     
    airport_names = list(data.columns)
    
    print(airport_names)
    
    start = input("Veuillez saisir l'airport de départ : ")
    # on suprime les distance > threshold remplacé par l'inf
    adjency_list = Graph_ville(data, threshold=1000)
    end = input("Veuillez saisir l'airport d'arrivée : ")
        
        
    return Dijkstra_path(adjency_list,start,end)

In [31]:
Dijkstra_city_path_user(data_villes)

['Athenes', 'Belgrade', 'Berlin', 'Berne', 'Bratislava', 'Bruxelles', 'Bucarest', 'Budapest', 'Chisinau', 'Copenhague', 'Dublin', 'Helsinki', 'La Haye', 'La Valette', 'Lisbonne', 'Londres', 'Ljubljana', 'Luxembourg', 'Madrid', 'Nicosie', 'Oslo', 'Paris', 'Podgorica', 'Prague', 'Reykjavik', 'Riga', 'Rome', 'Sarajevo', 'Sofia', 'Stockholm', 'Tallinn', 'Varsovie', 'Vienne', 'Zagreb']
Veuillez saisir l'airport de départ : Londres
Veuillez saisir l'airport d'arrivée : Nicosie


(['Londres', 'Luxembourg', 'Ljubljana', 'Podgorica', 'Athenes', 'Nicosie'],
 3320)

#### On obtient la même chose que Deumoucron entre Athenes et Stockholm

In [32]:
Dijkstra_city_path_user(data_villes)

['Athenes', 'Belgrade', 'Berlin', 'Berne', 'Bratislava', 'Bruxelles', 'Bucarest', 'Budapest', 'Chisinau', 'Copenhague', 'Dublin', 'Helsinki', 'La Haye', 'La Valette', 'Lisbonne', 'Londres', 'Ljubljana', 'Luxembourg', 'Madrid', 'Nicosie', 'Oslo', 'Paris', 'Podgorica', 'Prague', 'Reykjavik', 'Riga', 'Rome', 'Sarajevo', 'Sofia', 'Stockholm', 'Tallinn', 'Varsovie', 'Vienne', 'Zagreb']
Veuillez saisir l'airport de départ : Athenes
Veuillez saisir l'airport d'arrivée : Stockholm


(['Athenes', 'Belgrade', 'Varsovie', 'Stockholm'], 2443)

<div class="alert alert-block alert-warning"><h2>III - Algorithme d'affectation</h2>
    <ol>
        <li>Méthode hongroise</li>
    </ol>
</div>

# Méthode hongroise

In [33]:
test1 = np.array([[24,10, 21, 11],
              [14, 22, 10, 15],
              [15, 17, 20, 19],
              [11, 19, 14, 13]])

test2 = np.array([[17,15, 9, 5, 12],
              [16, 16, 10, 5,10],
              [12, 15, 14, 11, 5],
              [4, 8, 14, 17, 13],
              [13, 9, 8, 12, 17]])

<div class="alert alert-block alert-info">
    <h4>Cette fonction correspond aux deux premières étapes de la méthode hongroise la réduction des lignes et colonnes</h4>
    <hr style = "height:0.7px; border-width:0; color:gray; background-color:gray"/>
    <p>L'algorithme prend en parametre la matrice de coût</p>
    <p>Le principe de l'algorithme est le suivant : </p>
    <ol>
        <li>A chaque ligne on enleve son plus petit élément</li>
        <li>A chaque colonne on enleve son plus petit élément</li>
        <li>On retourne ensuite la matrice réduite</li>
    </ol>
</div>

In [34]:
import copy

In [35]:
def reduce_row_col(cost_matrix):
    reduced = copy.deepcopy(cost_matrix)
    n_row, n_col = cost_matrix.shape
    
    for row in range(n_row):
        r_min = np.min(reduced[row,:])
        reduced[row,:] = reduced[row,:] - r_min
        
    for col in range(n_col):
        c_min = np.min(reduced[:,col])
        reduced[:,col] = reduced[:,col] - c_min
    
    return reduced

In [36]:
test1_reduced = reduce_row_col(test1)
test1_reduced

array([[14,  0, 11,  0],
       [ 4, 12,  0,  4],
       [ 0,  2,  5,  3],
       [ 0,  8,  3,  1]])

<div class="alert alert-block alert-info">
    <h4>Cette fonction correspond à l'étape de selection des zeros dans la méthode hongroise</h4>
    <hr style = "height:0.7px; border-width:0; color:gray; background-color:gray"/>
    <p>L'algorithme prend en parametre la matrice de coût réduite</p>
    <p>Le principe de l'algorithme est le suivant : </p>
    <ol>
        <li>On tri les lignes par ordre décroissant en fonction de leur nombre de 0</li>
        <li>On va traiter les lignes en suivant l'ordre du tri en commancant donc par celles ayant le plus petit nombre de 0</li>
        <li>On choisit le 0 le plus à gauche on le sélectionne</li>
        <li>On barre les autres 0 qui se trouvent sur la même ligne et la même colonne</li>
        <li>On renvoie une matrice d'assignement de 0 et de 1 de même taille que la matrice de cout et qui à l'emplacement de chaque 0 selectionnés met un 1 et un 0 partout ailleurs  </li>
    </ol>
    <p>Lignes couvertes et colonnes couvertes correspondent au lignes et colonnes possèdant un 0 selectionné on s'en sert pour barrer les 0 à chaques étapes et on s'en servira dans les prochaines fonctions</p>
</div>

In [37]:
def choose_zeros(cost_matrix):
    
    n_row, n_col = cost_matrix.shape
    nb_zeros_row = []
    
    # Comptage des 0 sur chaques lignes afin de commencer par celle qui en a le moins
    for row in range(n_row):
        nb_zeros_row.append((np.sum(cost_matrix[row] == 0), row)) 
    nb_zeros_row.sort(key = lambda x: x[0], reverse=False)
    
    
    lignes_couvertes = [False] * n_row
    colonnes_couvertes = [False] * n_col
    assignement = np.array([[0 for _ in range(n_col)] for _ in range(n_row)])
    
    for _,row in nb_zeros_row:
        for col in range(n_col):
            if cost_matrix[row][col] == 0 and not lignes_couvertes[row] and not colonnes_couvertes[col]:
                assignement[row][col] = 1
                lignes_couvertes[row] = True
                colonnes_couvertes[col] = True
            
    return assignement, lignes_couvertes

In [38]:
test1_reduced

array([[14,  0, 11,  0],
       [ 4, 12,  0,  4],
       [ 0,  2,  5,  3],
       [ 0,  8,  3,  1]])

In [39]:
assignement, lignes_couvertes = choose_zeros(test1_reduced)
assignement

array([[0, 1, 0, 0],
       [0, 0, 1, 0],
       [1, 0, 0, 0],
       [0, 0, 0, 0]])

<div class="alert alert-block alert-info">
    <h4>Cette fonction correspond à l'étape de barrages des lignes et colonnes</h4>
    <hr style = "height:0.7px; border-width:0; color:gray; background-color:gray"/>
    <p>L'algorithme prend en parametre la matrice de coût reduite, les assignement réalisés par la sélection des zéros ainsi que les lignes couvertes</p>
    <p>Le principe de l'algorithme est le suivant : </p>
    <ol>
        <li>On doit marquer les lignes ne possèdant aucun zéro sélectionné, pour cela il suffit de parcourir les lignes et si elle est non couverte alors elle ne contient aucun zero sélectionné</li>
        <li>On boucle</li>
        <li>Etape A : On marque les colonnes ayant un zero barré(non selectionné) sur une ligne marquée</li>
        <li>Etape B : On marque les lignes ayant un zero sélectionné sur un colonne marquée</li>
        <li>On répète les étapes A et B en boucle jusqu'a ce que nous ne puissions plus marquer ni ligne ni colonne</li>
    </ol>
</div>

In [40]:
def mark_rows_and_cols(cost_matrix, assignement, lignes_couvertes):
    marked_row = set()
    marked_col = set()
    
    # Marquer les lignes ne contenant aucun zéro encadré
    for i in range(len(lignes_couvertes)):
        if lignes_couvertes[i] == False:
            marked_row.add(i)

    modif = True
    while(modif):
        modif = False
             
        # Marquer toute colonne ayant un zéro barré sur une ligne marquée
        for i in range(len(cost_matrix)):
            for j in range(len(cost_matrix)):
                if cost_matrix[i][j] == 0 and assignement[i][j] == 0 and i in marked_row and j not in marked_col:
                    marked_col.add(j)
                    modif = True
                    
        # Marquer toute ligne ayant un zéro encadré dans une colonne marquée
        for i in range(len(cost_matrix)):
            for j in range(len(cost_matrix)):
                if cost_matrix[i][j] == 0 and assignement[i][j] == 1 and j in marked_col and i not in marked_row:
                    marked_row.add(i)
                    modif = True
        
    return marked_row, marked_col

In [41]:
marked_row, marked_col = mark_rows_and_cols(test1_reduced, assignement, lignes_couvertes)
print(f'Lignes marquées : {marked_row}')
print(f'Colonnes marquées : {marked_col}')

Lignes marquées : {2, 3}
Colonnes marquées : {0}


<div class="alert alert-block alert-info">
    <h4>Cette fonction correspond à l'étape d'ajout du minimum et de soustraction du minimum</h4>
    <hr style = "height:0.7px; border-width:0; color:gray; background-color:gray"/>
    <p>L'algorithme prend en parametre la matrice de coût reduite, les colonnes marquées et les ligne marquées</p>
    <p>Le principe de l'algorithme est le suivant : </p>
    <ol>
        <li>On initialise la liste des colonnes et lignes barrée (ce qui est différent du fait d'être marqué) <br>
        En effet on barre les colonnes marquées et on barre les lignes non marquées</li>
        <li>On cherche le minimum parmis les élements qui ne sont ni sur une ligne barrée ni sur une colonne barrée</li>
        <li>On ajoute ce minimum à toutes les cases de la matrices reduite où la ligne est barrée et la colonne est barrée</li>
        <li>On enlève ce minimum à toutes les cases de la matrices reduite qui sont sur une lignes non barrée et une colonne non barrée</li>
        <li>On retourne la matrice de cout reduite et modifiée</li>
    </ol>
</div>

In [42]:
def add_min_not_marked(cost_matrix, marked_row, marked_col):

    col_non_barre = [i for i in range(len(cost_matrix)) if i not in marked_col]
    ligne_barre = [i for i in range(len(cost_matrix)) if i not in marked_row ]
  
    # Initialisation du minimum 
    if col_non_barre == []:
        minimum = cost_matrix[min(marked_row), 1]
    else:
        minimum = cost_matrix[min(marked_row), col_non_barre[0]]
    
    # Recherche du minimum 
    for ligne in marked_row:
        for col in col_non_barre:
            if cost_matrix[ligne][col] < minimum:
                minimum = cost_matrix[ligne][col]
    
    # Ajout et soustraction du minimum
    for ligne in marked_row:
        for col in range(cost_matrix.shape[0]):
            if col not in marked_col:
                cost_matrix[ligne][col] -= minimum
                
    for ligne in ligne_barre:
        for col in marked_col:
            cost_matrix[ligne][col] += minimum
    return cost_matrix

In [43]:
test1_reduced

array([[14,  0, 11,  0],
       [ 4, 12,  0,  4],
       [ 0,  2,  5,  3],
       [ 0,  8,  3,  1]])

In [44]:
test1_reduced = add_min_not_marked(test1_reduced, marked_row, marked_col)
test1_reduced

array([[15,  0, 11,  0],
       [ 5, 12,  0,  4],
       [ 0,  1,  4,  2],
       [ 0,  7,  2,  0]])

<div class="alert alert-block alert-info">
    <h4>Cette fonction verifie si l'assigement est optimal</h4>
    <hr style = "height:0.7px; border-width:0; color:gray; background-color:gray"/>
    <p>L'algorithme prend en parametre la matrice de coût reduite ainsi que les assignements réalisés par le choix des zeros</p>
    <p>Le principe de l'algorithme est le suivant : </p>
    <ol>
        <li>Retourne True si on a un unique zero selectionné par lignes et par colonnes et False sinon</li>
    </ol>
</div>

In [45]:
def verif_solution(cost_matrix, assignement):
    return cost_matrix.shape[0] == np.sum(assignement.flatten())

In [46]:
verif_solution(test1_reduced,assignement)

False

In [47]:
assignement1, _ = choose_zeros(test1_reduced)
assignement1

array([[0, 1, 0, 0],
       [0, 0, 1, 0],
       [1, 0, 0, 0],
       [0, 0, 0, 1]])

In [48]:
verif_solution(test1_reduced,assignement1)

True

<div class="alert alert-block alert-info">
    <h4>Cette fonction est l'algorithme associé à la methode hongroise</h4>
    <hr style = "height:0.7px; border-width:0; color:gray; background-color:gray"/>
    <p>L'algorithme prend en parametre la matrice de coût initiale</p>
    <p>Le principe de l'algorithme est le suivant : </p>
    <ol>
        <li>On réduit la matrice de coût avec la fonction "reduce_row_col()"</li>
        <li>On boucle tant que la solution optimale n'est pas trouvée</li>
        <li>On assigne en choisissant les zeros avec la fonction "choose_zeros()"</li>
        <li>On marque les lignes et les colonnes avec la fonction "mark_rows_cols()"</li>
        <li>On effectue l'opération de soustraction et d'ajout du minimum avec la fonction "add_min_not_marked()"</li>
        <li>On réassigne avec "choose_zeros()"</li>
        <li>On verifie si la solution est optimale avec "verif_solution()"</li>
    </ol>
</div>

In [49]:
def hongroise(matrice):
    reduced = reduce_row_col(matrice)
    i = 0
    temp = np.zeros(matrice.shape)
    while(True):
        
        assignement, lignes_couvertes = choose_zeros(reduced)
        # if i%10 == 0:
        #    # assert(not(np.all(assignement == temp)))
        #     print(lignes_couvertes)
        if verif_solution(reduced, assignement):
            print(lignes_couvertes)
            return assignement
        
        marked_row, marked_col = mark_rows_and_cols(reduced, assignement, lignes_couvertes)
        reduced = add_min_not_marked(reduced, marked_row, marked_col)
        i+= 1
        temp = assignement
    return assignement

In [50]:
test1

array([[24, 10, 21, 11],
       [14, 22, 10, 15],
       [15, 17, 20, 19],
       [11, 19, 14, 13]])

In [51]:
hongroise(test1)

[True, True, True, True]


array([[0, 1, 0, 0],
       [0, 0, 1, 0],
       [1, 0, 0, 0],
       [0, 0, 0, 1]])

In [52]:
test2

array([[17, 15,  9,  5, 12],
       [16, 16, 10,  5, 10],
       [12, 15, 14, 11,  5],
       [ 4,  8, 14, 17, 13],
       [13,  9,  8, 12, 17]])

In [53]:
hongroise(test2)

[True, True, True, True, True]


array([[0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0]])

### Application aux 21 premières villes

<div class="alert alert-block alert-info">
    <h4>Cette fonction applique la méthode hongroise aux 21 premières villes</h4>
    <hr style = "height:0.7px; border-width:0; color:gray; background-color:gray"/>
    <p>L'algorithme prend en parametre le dataset des villes</p>
    <p>Le principe de l'algorithme est le suivant : </p>
    <ol>
        <li>On ajoute sur la diagonale une très grande valeure</li>
        <li>On renvoie l'affectation de chaques villes avec la fonction "hongroise()"</li>
    </ol>
    <p>Malheureusement le programme ne marche pas sur le dataset entier au dessus de 21 villes le programme boucle à l'infini car on obtient jamais de solution optimale. Les afectations ne varient plus d'un tour à l'autre et 5 lignes restent sans aucune affectation. Malgré ca il fonctionne bien sur les deux tests vu précédement </p>
</div>

In [54]:
def hongroise_city_assignement(data, coupe = 21):
    inf = np.inf
    data_v = np.array(data)
    data_v = data_v + 1000000 * np.eye(data_v.shape[0])
    
    return hongroise(data_v[:coupe, :coupe])

In [55]:
hongroise_city_assignement(data_villes)

[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]


array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

<div class="alert alert-block alert-warning"><h2>IV - Heuristiques du voyageur de commerce</h2>
    <ol>
        <li>Méthode du plus proche voisin</li>
        <li>Méthode de l'insertion de cout minimal</li>
        <li>Regret maximal</li>
    </ol>
</div>

# Méthode du plus proche voisin

In [56]:
adjacency_list = {'A': {'B': 1, 'C': 1, 'D' : 2},
                'B': {'A': 1, 'D' : 1, 'C' : 2},
                'C' : {'A': 1, 'D' : 10, 'B' : 2},
                'D' : {'A': 2, 'B' : 1, 'C' : 10}
                  }

<div class="alert alert-block alert-info">
    <h4>Cette fonction correspond à l'heuristique 'du plus proche voisin' pour répondre au probleme du voyageur de commerce </h4>
    <hr style = "height:0.7px; border-width:0; color:gray; background-color:gray"/>
    <p>L'algorithme prend en parametre la liste d'adjacence du graphe ainsi qu'un noeud de départ</p>
    <p>Le principe de l'algo est le suivant : </p>
    <ol>
        <li>On bloucle tant qu'on a pas parcouru tous les noeuds</li>
        <li>On enregistre dans voisins tous les voisins du noeud en cours de traitement. Il ne faut pas qu'ils soient egaux au noeud de départ (noeud de fin) et il ne faut pas qu'ils aient déja été traité</li>
        <li>On choisi le voisin le plus proche. Si il y a égalité entre deux noeux on tire aléatoirement avec la fonction "choice()" de "random" on pourrait prendre le premier élément automatiquement car "minis_voisins" contiens tous les plus proches voisins</li>
        <li>On traite le noeud que l'on vient de choisir</li>
        <li>Si la liste de voisins est vide alors on est obligé de rentrer au noeud de départ</li>
    </ol>
</div>

In [57]:
from random import *

In [58]:
def nearest_neighbor(liste_adjacence, start):
    end = start
    actuelle = start
    n = len(liste_adjacence.keys())
    res = []
    val = 0
    while(len(res) != n):
        res.append(actuelle)
        
        voisins = {key: val for key, val in liste_adjacence[actuelle].items() if key != end and key not in res}
        
        # On cherche les élément minimaux 
        minis_voisins =  [(key,val) for key,val in voisins.items() if all(voisins[temp] >= voisins[key] for temp in voisins)]
        if voisins == {}:
            res.append(end)
            val += liste_adjacence[actuelle][end]
            return res, val
        temp = choice(minis_voisins)
        actuelle = temp[0]
        val += temp[1]
        
        
    res.append(end)
    return res, val

#### C'est une heuristique ici on obtient un chemin de longueur 13 alors que le chemin optimal est [B,D,A,C,B] de longueur 6 ceci dépend du choix aléatoire de l'arc de coup minimal

In [61]:
nearest_neighbor(adjacency_list, 'B')

(['B', 'D', 'A', 'C', 'B'], 6)

In [62]:
nearest_neighbor(adjacency_list, 'A')

(['A', 'B', 'D', 'C', 'A'], 13)

### Application au dataset des villes

In [63]:
liste = Graph_ville(data_villes, threshold=10000000)

In [64]:
nearest_neighbor(liste, 'Athenes')

(['Athenes',
  'Sofia',
  'Bucarest',
  'Chisinau',
  'Belgrade',
  'Sarajevo',
  'Podgorica',
  'Zagreb',
  'Ljubljana',
  'Vienne',
  'Bratislava',
  'Budapest',
  'Prague',
  'Berlin',
  'Copenhague',
  'Oslo',
  'Stockholm',
  'Tallinn',
  'Helsinki',
  'Riga',
  'Varsovie',
  'Luxembourg',
  'Bruxelles',
  'La Haye',
  'Londres',
  'Paris',
  'Berne',
  'Rome',
  'La Valette',
  'Madrid',
  'Lisbonne',
  'Dublin',
  'Reykjavik',
  'Nicosie',
  'Athenes'],
 21610)

# Méthode de l'insertion de cout minimal

<div class="alert alert-block alert-info">
    <h4>Cette fonction permet de calculer le cout d'un chemin</h4>
    <hr style = "height:0.7px; border-width:0; color:gray; background-color:gray"/>
    <p>L'algorithme prend en parametre la liste d'adjacence du graphe ainsi qu'une liste de noeuds</p>
    <p>Le principe de l'algorithme est le suivant : </p>
    <ol>
        <li>On parcours la liste de noeuds et on regarde la distance entre le noeud actuelle et le noeud suivant à l'aide de la liste d'adjacence</li>
    </ol>
</div>

In [65]:
def compute_cost(liste_adjacence, liste_noeud):
    res = 0
    for node in range(len(liste_noeud) - 1):
        res += liste_adjacence[liste_noeud[node]][liste_noeud[node+1]]
    return res

In [66]:
compute_cost(adjacency_list, ['B', 'A', 'C', 'D', 'B'])

13

<div class="alert alert-block alert-info">
    <h4>Cette fonction permet de trouver l'insertion optimale</h4>
    <hr style = "height:0.7px; border-width:0; color:gray; background-color:gray"/>
    <p>L'algorithme prend en parametre la liste d'adjacence du graphe, une liste de noeud ainsi qu'un autre noeud</p>
    <p>Le principe de l'algo est le suivant : </p>
    <ol>
        <p>Le but est d'inserer le nouveau noeud à la bonne place dans la liste afin de minimiser le cout total de la liste</p>
        <li>On parcourt avec i toute les positions possible du noeud</li>
        <li>On insere le noeud à la position i et on calcul le cout du chemin avec "compute_cost()"</li>
        <li>On store le tuple (nouveau chemin, cout du chemin) dans un liste "set_res"</li>
        <li>On tri par ordre croissant de cout de chemin la liste de tuple "set_res"</li>
        <li>On pourra donc prendre le premier élement de la liste "set_res" qui sera donc l'une des insertion optimale</li>
        <p>On pourrait aussi tirer aléatoirement parmis les insertions optimales</p>
    </ol>
</div>

In [67]:
def placement_liste(liste_noeud, element, liste_adjacence):
    set_res = []
    val = 0
    res = liste_noeud.copy()
    for i in range(len(liste_noeud)+1):
        res = liste_noeud.copy()
        res.insert(i, element)
        val = compute_cost(liste_adjacence, res)
        set_res.append((res, val))
    set_res.sort(key = lambda x: x[1], reverse=False) 
    return set_res
        

In [68]:
placement_liste(['A', 'C'], 'B', adjacency_list)

[(['B', 'A', 'C'], 2), (['A', 'B', 'C'], 3), (['A', 'C', 'B'], 3)]

<div class="alert alert-block alert-info">
    <h4>Cette fonction permet de trouver l'insertion optimale dans le cas où il ne reste que un noeud à traiter</h4>
    <hr style = "height:0.7px; border-width:0; color:gray; background-color:gray"/>
    <p>L'algorithme prend en parametre la liste d'adjacence du graphe, une liste de noeud ainsi qu'un autre noeud</p>
    <p>Le principe de l'algorithme est le suivant : </p>
    <ol>
        <p>Le but est d'inserer le nouveau noeud à la bonne place dans la liste afin de minimiser le cout total de la liste <strong>en prenant en compte que c'est un cycle donc un chemin qui revient au point de départ</strong></p>
        <li>On parcourt avec i toute les position possible du noeud</li>
        <li>On insert le noeud à la position i </li>
        <li><strong>On ajoute le premier élement de la liste à la fin afin d'avoir un cycle</strong></li>
        <li>On calcul le cout du chemin avec "compute_cost()"</li>
        <li>On store le tuple (nouveau chemin, cout du chemin) dans un liste "set_res"</li>
        <li>On tri par ordre croissant de cout de chemin la liste de tuples "set_res"</li>
        <li>On pourra donc prendre le premier élement de la liste "set_res" qui sera donc l'une des insertion optimale</li>
        <p>On pourrait aussi tirer aléatoirement parmis les insertions optimales</p>
    </ol>
</div>

In [69]:
def placement_liste_final(liste_noeud, element, liste_adjacence):
    set_res = []
    res = liste_noeud.copy()
    for i in range(len(liste_noeud)+1):
        res = liste_noeud.copy()
        res.insert(i, element)
        res.append(res[0])
        val = compute_cost(liste_adjacence, res)
        set_res.append((res, val))
    set_res.sort(key = lambda x: x[1], reverse=False) 
    return set_res

In [70]:
placement_liste_final(['D', 'B', 'A'], 'C', adjacency_list)

[(['D', 'B', 'C', 'A', 'D'], 6),
 (['C', 'D', 'B', 'A', 'C'], 13),
 (['D', 'B', 'A', 'C', 'D'], 13),
 (['D', 'C', 'B', 'A', 'D'], 15)]

<div class="alert alert-block alert-info">
    <h4>Cette fonction correspond à l'heuristique "insertion de cout minimal" pour répondre au probleme du voyageur de commerce </h4>
    <hr style = "height:0.7px; border-width:0; color:gray; background-color:gray"/>
    <p>L'algorithme prend en parametre la liste d'adjacence du graphe ainsi qu'un noeud de départ</p>
    <p>Le principe de l'algo est le suivant : </p>
    <ol>
        <li>On bloucle tant qu'on a pas parcouru tous les noeuds</li>
        <li>On récupère dans une liste "voisins" tous les noeuds adjacents a un moins un des noeud du chemin "res" si et seulement s'il n'est pas déja dans les voisins et si il n'est pas deja dans le chemin</li>
        <li>Pour chaque voisin on va récuperer son placement optimal dans le chemin avec "placement_liste()[0]" et on le store dans une list "minis_voisins"</li>
        <li>On a donc pour chaques voisins l'un de ses placement optimal, on tri ensuite la liste "minis_voisins" dans l'odre croissant des couts des placements optimaux. On a donc en premier l'un des placement optimal de l'un des voisins</li>
        <li>Notre nouveau chemin sera donc ce placement optimal</li>
        <li>Si la longeure du chemin est égale au nombre de noeuds du graphe moins 1, autement dit s' il ne reste plus qu'un seul noeud à traiter, nous devons pour le calcul du placement optimal prendre en compte que le chemin doit être un cycle. On doit donc prendre en compte le noeud de départ dans le placement optimal. On utilise alors la fonction "placement_liste_final()"</li>      </ol>
</div>

In [71]:
def insertion_minimale(liste_adjacence, start):
    seen = []
    res = [start]
    
    n = len(liste_adjacence)
    i = 0
    
    while(len(res)!= n):
        voisins = []
        for node in res:
            for voisin in liste_adjacence[node]:
                if voisin not in seen and voisin not in voisins:
                    voisins.append(voisin)
        minis_voisin = []
        for voisin in voisins:
            
            minis_voisin.append(placement_liste(res, voisin, liste_adjacence)[0])
        
        #print(minis_voisin)
        
        if len(res) == n-1:

            res = placement_liste_final(res, str(voisin), liste_adjacence)[0]
            return res
        
        minis_voisin.sort(key = lambda x: x[1], reverse=False)
        
        
        res = minis_voisin[0][0]
        seen = res
        
        i+=1 
    return res
        
    

In [72]:
 insertion_minimale(adjacency_list, 'A')

(['D', 'B', 'C', 'A', 'D'], 6)

In [73]:
insertion_minimale(liste, 'Athenes')

(['Nicosie',
  'Chisinau',
  'Bucarest',
  'Sofia',
  'Belgrade',
  'Budapest',
  'Bratislava',
  'Vienne',
  'Varsovie',
  'Riga',
  'Tallinn',
  'Helsinki',
  'Stockholm',
  'Oslo',
  'Copenhague',
  'Berlin',
  'Prague',
  'Zagreb',
  'Ljubljana',
  'Luxembourg',
  'La Haye',
  'Bruxelles',
  'Londres',
  'Reykjavik',
  'Dublin',
  'Paris',
  'Berne',
  'Rome',
  'Sarajevo',
  'Podgorica',
  'Athenes',
  'La Valette',
  'Madrid',
  'Lisbonne',
  'Nicosie'],
 21788)